In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from tqdm import tqdm

# Image Panorama Stitching

In [ ]:
img1 = cv2.imread('img1.jpg')
img2 = cv2.imread('img2.jpg')

In [ ]:
fig = plt.figure(figsize=(6.4 * 2, 4.8 * 2))

plt.subplot(121)
plt.imshow(img1,)

plt.subplot(122)
plt.imshow(img2,)

fig.tight_layout()
plt.show()

In [ ]:
descriptor = cv2.xfeatures2d.SIFT_create()

In [ ]:
kp1, desc1 = descriptor.detectAndCompute(img1, None)
kp2, desc2 = descriptor.detectAndCompute(img2, None)

kNN mathcer

![alt text](knn_example.png "Title")

In [ ]:
FLANN_INDEX_KDTREE = 2
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

# flann = cv2.FlannBasedMatcher(index_params, search_params)
flann = cv2.BFMatcher()

In [ ]:
matches = flann.match(desc2, desc1)
print(matches[0].distance)

matches = flann.knnMatch(desc2, desc1, k=2)
print([m.distance for m in matches[0]])

In [ ]:
if (desc1 is not None and desc2 is not None and len(desc1) >=2 and len(desc2) >= 2):
    rawMatch = flann.knnMatch(desc2, desc1, k=2)
    matches = []
    # ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
    ratio = 0.75
    for m in rawMatch:
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append(m[0])

In [ ]:
list_kp1 = np.array([kp1[mat.trainIdx].pt for mat in matches])
list_kp2 = np.array([kp2[mat.queryIdx].pt for mat in matches])

In [ ]:
if len(matches) > 4:
    (H, status) = cv2.findHomography(list_kp1, list_kp2, cv2.RANSAC)

In [ ]:
# Apply panorama correction
width = img1.shape[1] + img2.shape[1]
height = img2.shape[0] + img2.shape[0]

result = cv2.warpPerspective(img1, H, (width, height))

result[0:img2.shape[0], 0:img2.shape[1]] = img2
idx_w = np.argwhere(result[0, :] == 0)
idx_h = np.argwhere(result[:, 0] == 0)

plt.figure(figsize=(20,10))
plt.imshow(result)

plt.axis('off')
plt.show()

# Pazzle

## Step by step 

In [ ]:
def get_images(path="puzzle/china_shuffle"):
    images_path = os.listdir(path)
    images = []
    for im in images_path:
        img = cv2.imread(os.path.join(path, im))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        images.append(img)

    return np.array(images)

In [ ]:
path = 'puzzle/china'
images = get_images(path)[3:5]
print(images.shape)

In [ ]:
img1, img2 = images[0], images[1]

sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=False)

print(des1.shape, des2.shape)

In [ ]:
matches = bf.match(des2, des1)

for i in range(3):
    m = matches[i]
    line = 'match info: \n1) distance: {}\n2) imgIdx: {}\n3) queryIdx: {}\n4) trainIdx: {}'.format(m.distance, m.imgIdx, m.queryIdx, m.trainIdx)
    print(line)

In [ ]:
matches = sorted(matches, key=lambda x: x.distance)
good_matches = matches[:10]

In [ ]:
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1] + img2.shape[1], 3), dtype=np.uint8)

draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
                  )

cv2.drawMatches(img1, kp1, img2, kp2, good_matches, img_matches, **draw_params)

#-- Show detected matches
plt.imshow(img_matches)

In [ ]:
list_kp1 = np.array([kp1[mat.trainIdx].pt for mat in good_matches])
list_kp2 = np.array([kp2[mat.queryIdx].pt for mat in good_matches])

print(list_kp1)

In [ ]:
x, y = list_kp1[:, 0] - list_kp2[:, 0], list_kp1[:, 1] - list_kp2[:, 1]
angles = [math.degrees(math.atan2(y[elem], x[elem])) for elem in range(len(x))]

np.std(angles)

In [ ]:
diff_x, diff_y = int(np.mean(x)), int(np.mean(y))

img3 = np.zeros((max(img1.shape[0], abs(diff_y) + img2.shape[0], img1.shape[0] - diff_y),
                 max((img1.shape[1], abs(diff_x) + img2.shape[1], img1.shape[1] - diff_x))))

plt.imshow(img3)

In [ ]:
diff_x1, diff_y1 = max(0, -diff_x), max(0, -diff_y)
diff_x2, diff_y2 = max(0, diff_x), max(0, diff_y)

print(diff_x1, diff_y1)
print(diff_x2, diff_y2)

img3[diff_y1:diff_y1 + img1.shape[0], diff_x1:diff_x1 + img1.shape[1]] = img1
img3[diff_y2:diff_y2 + img2.shape[0], diff_x2:diff_x2 + img2.shape[1]] = img2

In [ ]:
fig = plt.figure(figsize=(6.4 * 2, 4.8 * 2))

plt.subplot(131)
plt.imshow(img3, cmap='gray')

plt.subplot(132)
plt.imshow(img1, cmap='gray')

plt.subplot(133)
plt.imshow(img2, cmap='gray')

fig.tight_layout()
plt.show()

## All solution

In [ ]:
def stitcher(img1, img2, match_number=20, best_match_number=5, angle_eps=2):
    img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=False)

    if des2 is not None and des1 is not None:
        matches = bf.match(des2, des1)
        matches = sorted(matches, key=lambda x: x.distance)
        if len(matches) > match_number:
            matches = matches[:best_match_number]
            list_kp1 = np.array([kp1[mat.trainIdx].pt for mat in matches])
            list_kp2 = np.array([kp2[mat.queryIdx].pt for mat in matches])
            x, y = list_kp1[:, 0] - list_kp2[:, 0], list_kp1[:, 1] - list_kp2[:, 1]
            angles = [math.degrees(math.atan2(y[elem], x[elem])) for elem in range(len(x))]

            if np.std(angles) < angle_eps:
                diff_x, diff_y = int(np.mean(x)), int(np.mean(y))
                img3 = np.zeros((max(img1.shape[0], abs(diff_y) + img2.shape[0], img1.shape[0] - diff_y),
                                 max((img1.shape[1], abs(diff_x) + img2.shape[1], img1.shape[1] - diff_x))))
                diff_x1, diff_y1 = max(0, -diff_x), max(0, -diff_y)
                diff_x2, diff_y2 = max(0, diff_x), max(0, diff_y)

                img3[diff_y1:diff_y1 + img1.shape[0], diff_x1:diff_x1 + img1.shape[1]] = img1
                img3[diff_y2:diff_y2 + img2.shape[0], diff_x2:diff_x2 + img2.shape[1]] = img2
                return img3

In [ ]:
path = 'puzzle/china_shuffle'
images = get_images(path)
random_idxs = np.arange(0, len(images))
np.random.shuffle(random_idxs)
# images = images[random_idxs]

In [ ]:
res = images[0]
images = np.delete(images, 0, 0)
count = 0

pbar = tqdm(total=150)
while len(images) >= 1 and count <= 150:
    idx = count % len(images)
    count += 1
    cur = stitcher(res, images[idx])  # match_number=20 // (k + 1), best_match_number=5 + 3 * k, angle_eps=2 + 5 * k)
    if cur is not None:
        res = cur
        images = np.delete(images, idx, 0)
        
    pbar.update()

In [ ]:
fig = plt.figure(figsize=(6.4 * 2, 4.8 * 2))

plt.imshow(res)

fig.tight_layout()
plt.show()